In [44]:
import pandas as pd
import numpy as np

In [45]:
ratings = pd.read_csv('ml-ratings.csv')

In [46]:
preds = pd.read_csv('ml-preds.csv')

In [4]:
ratings

,Unnamed: 0,userId,itemId,rating
0,0,1,6,5
1,1,1,10,3
2,2,1,12,5
3,3,1,14,5
4,4,1,17,3
...,...,...,...,...
17914,19937,450,1160,5
17915,19938,450,1212,4
17916,19939,450,1226,4
17917,19940,450,1263,4


# 1. 전처리

In [5]:
ratings.head()

,Unnamed: 0,userId,itemId,rating
0,0,1,6,5
1,1,1,10,3
2,2,1,12,5
3,3,1,14,5
4,4,1,17,3


In [6]:
ratings_id = ratings.userId.unique()
pred_id = preds.userId.unique()

In [25]:
ratings_id

array([  1,   2,   3,   5,   6,   7,   8,  10,  11,  12,  13,  14,  15,
        16,  18,  20,  21,  22,  23,  24,  25,  26,  28,  37,  38,  41,
        42,  43,  44,  48,  49,  52,  54,  56,  57,  58,  59,  60,  62,
        63,  64,  65,  69,  70,  72,  73,  75,  76,  77,  79,  81,  82,
        83,  84,  85,  87,  89,  90,  91,  92,  94,  95,  96,  97,  99,
       100, 101, 102, 104, 106, 109, 110, 112, 113, 114, 115, 116, 117,
       118, 119, 121, 122, 123, 125, 126, 128, 130, 135, 137, 138, 141,
       144, 145, 148, 150, 151, 152, 154, 157, 158, 159, 160, 161, 162,
       164, 167, 168, 174, 176, 177, 178, 180, 181, 183, 184, 185, 186,
       187, 188, 189, 190, 193, 194, 195, 197, 198, 199, 200, 201, 203,
       204, 206, 207, 208, 210, 213, 214, 215, 216, 217, 218, 221, 222,
       223, 224, 227, 230, 232, 233, 234, 235, 236, 237, 239, 243, 244,
       246, 248, 249, 250, 251, 253, 254, 255, 256, 257, 262, 263, 264,
       267, 268, 269, 270, 271, 272, 274, 275, 276, 277, 279, 28

In [49]:
ratings['rank'] = ratings.groupby('userId').rating.rank(ascending = False, method = 'first')
preds['rank'] = preds.groupby('userId').rating.rank(ascending = False, method = 'first')

,Unnamed: 0,userId,itemId,rating,rank
0,0,1,6,5,1.0
1,1,1,10,3,80.0
2,2,1,12,5,2.0
3,3,1,14,5,3.0
4,4,1,17,3,81.0
...,...,...,...,...,...
17914,19937,450,1160,5,14.0
17915,19938,450,1212,4,32.0
17916,19939,450,1226,4,33.0
17917,19940,450,1263,4,34.0


## @10 sheet 제작

In [9]:
def make_k_sheet(data, k):
    tmp_list = data.userId.unique()
    for num in range(len(tmp_list)):
        if num == 0 :
            tmp_df = data[data.userId == int(tmp_list[num])][:k]
        else:
            tmp_df = tmp_df.append(data[data.userId == int(tmp_list[num])][:k])
    return tmp_df

In [28]:
ratings10 = make_k_sheet(ratings,10)
preds10 = make_k_sheet(preds,10)

In [36]:
preds_all = preds.copy()
preds_all.columns = ['Unnamed: 0', 'userId', 'itemId', 'pred_rating', 'pred_rank']

In [29]:
preds10.columns = ['Unnamed: 0', 'userId', 'itemId', 'pred_rating', 'pred_rank']

In [30]:
preds10 = preds10.drop(columns=['Unnamed: 0'])

In [47]:
result10 = ratings.merge(preds, how='left', on=['userId', 'itemId'])

In [48]:
result10[result10.userId == 1].head()

,Unnamed: 0_x,userId,itemId,rating_x,Unnamed: 0_y,rating_y
0,0,1,6,5,0,3.846799
1,1,1,10,3,1,3.997140
2,2,1,12,5,2,4.567387
3,3,1,14,5,3,3.943916
4,4,1,17,3,4,3.097442


In [33]:
result10.count()

Unnamed: 0     2670
userId         2670
itemId         2670
rating         2670
rank           2670
pred_rating     838
pred_rank       838
dtype: int64

In [34]:
838/2670

0.3138576779026217

In [14]:
all_recall = result10.groupby('userId')['pred_rating'].count()/10
all_recall

userId
1      0.0
2      0.6
3      0.5
5      0.2
6      0.1
      ... 
425    0.4
429    0.3
435    0.6
437    0.5
450    0.4
Name: pred_rating, Length: 267, dtype: float64

In [15]:
all_recall.sum()/len(all_recall)

0.31385767790262176

## MRR

In [16]:
result10.index = [ x for x in range(1,11)]*267

In [17]:
result10.head()

,Unnamed: 0,userId,itemId,rating,rank,pred_rating,pred_rank
1,19937,450,1160,5,46.0,NaN,NaN
2,19933,450,942,5,45.0,NaN,NaN
3,19923,450,659,5,44.0,4.413108,43.0
4,19919,450,496,5,43.0,4.407806,42.0
5,19916,450,470,5,42.0,NaN,NaN


In [18]:
all_mrr = result10[result10.pred_rating.isnull() != True]
all_mrr

,Unnamed: 0,userId,itemId,rating,rank,pred_rating,pred_rank
3,19923,450,659,5,44.0,4.413108,43.0
4,19919,450,496,5,43.0,4.407806,42.0
8,19909,450,216,5,39.0,4.332774,39.0
9,19908,450,194,5,38.0,4.863709,45.0
3,19813,437,499,5,23.0,4.113330,22.0
...,...,...,...,...,...,...,...
2,155,2,313,5,21.0,4.257496,19.0
3,140,2,251,5,20.0,4.361705,21.0
4,139,2,50,5,19.0,4.493536,22.0
5,151,2,303,4,18.0,3.813972,14.0


In [19]:
tmp_list = []
tmp_userId = all_mrr.userId.unique()

In [20]:
for i in range(len(tmp_userId)):
    tmp_num = all_mrr[all_mrr.userId == int(tmp_userId[i])].index[0]
    tmp_list = tmp_list + [1/tmp_num]

In [21]:
mrr_df = pd.DataFrame({'userId': all_mrr.userId.unique(), 'RR': tmp_list})
mrr_df

,userId,RR
0,450,0.333333
1,437,0.333333
2,435,0.500000
3,429,0.500000
4,425,1.000000
...,...,...
241,7,0.333333
242,6,0.125000
243,5,0.111111
244,3,1.000000


In [22]:
mrr_df.RR.sum()/len(mrr_df)

0.4521712479029552